# Welcome to the Prompt engineering Workshop
Lets first install the pre-requite python packages

In [ ]:
# pandas is a library for parsing csvs
!pip install pandas
# also as we will be using claude we need to install anthropic
!pip install anthropic

We will generating code for parsing some csv so lets create the csv to work with quickly

In [2]:
import csv

# For Workshop purpose so that everyone can acess the same data, we writing the file dynamically 
import csv

# Create data as list of lists
data = [
    ["Flow ID","Tx Frames","Rx Frames","Frames Delta","Loss %","Tx Frame Rate","Rx Frame Rate","Tx L1 Rate (bps)","Rx L1 Rate (bps)","Rx Bytes","Tx Rate (Bps)","Rx Rate (Bps)","Tx Rate (bps)","Rx Rate (bps)","Tx Rate (Kbps)","Rx Rate (Kbps)","Tx Rate (Mbps)","Rx Rate (Mbps)","Store-Forward Avg Latency (ns)","Store-Forward Min Latency (ns)","Store-Forward Max Latency (ns)","First TimeStamp","Last TimeStamp"],
    ["Flow 1",1000,1000,0,0.000,0.000,0.000,0.000,0.000,66000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,"00:00:01.063","00:00:04.393"],
    ["Flow 2",1000,100,0,0.000,0.000,0.000,0.000,0.000,66000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,"00:00:01.063","00:00:04.393"],
    ["Flow 3",1000,1500,0,0.000,0.000,0.000,0.000,0.000,66000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,"00:00:01.063","00:00:04.393"],
    ["Flow 4",1000,"N/A",0,0.000,0.000,0.000,0.000,0.000,66000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0,0,0,"00:00:01.063","00:00:04.393"]
]

# Write to CSV file
with open('stats.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)


In [ ]:
# check the file is created
!cat stats.csv

## Lets get started with our first prompt
### Level 1: The "Zero-Shot"
Asking the model to perform a task with no examples, no context, and no constraints. You are relying entirely on the model's training data assumptions.

In [ ]:
from anthropic import AnthropicFoundry
import re

def extract_and_save_python_code(text, output_file='output.py'):
    # Pattern to match ```python ... ``` blocks
    pattern = r'```python\s*(.*?)```'
    
    # Find all matches (re.DOTALL allows . to match newlines)
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Join all code blocks if multiple exist
    python_code = '\n\n'.join(matches)
    
    # Write to file
    with open(output_file, 'w') as f:
        f.write(python_code)
    
    return python_code

# API Configuration
endpoint = "https://foundary-codegen-1.services.ai.azure.com/anthropic/"
deployment_name = "claude-sonnet-4-5"
api_key = ""

client = AnthropicFoundry(
    api_key=api_key,
    base_url=endpoint
)

prompt = """
Write a python script to read a csv file called 'stats.csv' 
calculate the total loss by looking at Tx Frames and Rx Frames.
I want you to calculate loss as (rx - tx) / tx * 100 and print the result for all Flow 2, Flow3 and Flow 4.
we will havve multiple columns but the relevant columns are Flow ID, Tx Frames, Rx Frames.
return only the python code without any explanation or comments.
"""

message = client.messages.create(
    model=deployment_name,
    messages=[
        {"role": "user", "content": prompt}
    ],
    max_tokens=1024,
)

print(message.content[0].text)
extract_and_save_python_code(message.content[0].text, output_file='traffic_loss_calculator.py')


In [ ]:
# lets run the python code generated by the model
!python traffic_loss_calculator.py